In [1]:
import torch
import numpy as np
import os
import matplotlib.pyplot as plt

from model.hnn import HNN
#from dynamics_library import double_well  # Replace with your actual import

In [2]:
# ==== Load the trained full model ====
model_path = "models/model_0_double_well_0.01_im.pt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load(model_path, map_location=device)
model.eval()


/tmp/ipykernel_22144/2504873418.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


HNN(
  (differentiable_model): MLP(
    (layer0): Linear(in_features=2, out_features=200, bias=True)
    (layer1): Linear(in_features=200, out_features=200, bias=True)
    (layer2): Linear(in_features=200, out_features=1, bias=False)
  )
)

In [14]:
# ==== Load the trained full model ====
model_path = "models/model_0_coupled_ho_0.01_im.pt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load(model_path, map_location=device)
model.eval()


HNN(
  (differentiable_model): MLP(
    (layer0): Linear(in_features=2, out_features=200, bias=True)
    (layer1): Linear(in_features=200, out_features=200, bias=True)
    (layer2): Linear(in_features=200, out_features=1, bias=False)
  )
)

In [21]:
# ==== Define phase space grid ====
num_pts = 30
q_vals = np.linspace(-3, 3, num_pts)
p_vals = np.linspace(-3, 3, num_pts)
Q, P = np.meshgrid(q_vals, p_vals)

# ==== Containers ====
pred_h = np.zeros((num_pts, num_pts))
true_h = np.zeros((num_pts, num_pts))

In [22]:
# ==== Compute true and predicted Hamiltonians ====
for i in range(num_pts):
    coords = torch.tensor([Q[i, :], P[i, :]], dtype=torch.float32).T.to(device)

    # True Hamiltonian (Double Well)
    #true_h[i, :] = (0.5 * P[i, :]**2 + 0.25 * Q[i, :]**4 - 0.5 * Q[i, :]**2)

    #Coupled HO
    alpha = 0.5
    #true_dyn = coupled_ho(torch.tensor([Q[i,:],P[i,:]], dtype=torch.float32).T, args, None).detach().numpy()
    true_h[i,:] = Q[i,:]**2 + P[i,:]**2 + alpha*Q[i,:]*P[i,:]

    # Predicted Hamiltonian
    with torch.no_grad():
        pred_h[i, :] = model(coords).cpu().numpy()

# ==== Calculate Offset ====
offset = (pred_h - true_h).mean()
print(f"Computed offset: {offset:.4f}")


Computed offset: -38.7906


In [23]:
# ==== Adjust predictions ====
pred_h_corrected = pred_h - offset

In [24]:
# Absolute error
abs_error = np.abs(pred_h_corrected - true_h)
mean_abs_error = abs_error.mean()

In [25]:
mean_abs_error

1.6902081242254285

In [ ]:
# ==== Optional: Save or visualize ====
fig = plt.figure(figsize=(12, 5))

ax1 = fig.add_subplot(1, 2, 1, projection='3d')
ax1.plot_surface(Q, P, true_h, cmap='viridis')
ax1.set_title("True Hamiltonian")

ax2 = fig.add_subplot(1, 2, 2, projection='3d')
ax2.plot_surface(Q, P, pred_h_corrected, cmap='plasma')
ax2.set_title("Predicted Hamiltonian (offset corrected)")

plt.tight_layout()
plt.show()

In [ ]:
#########For Henon-Heiles##########

# ==== Load the trained full model ====
model_path = "models/model_0_henon_heiles_0.01_im.pt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load(model_path, map_location=device)
model.eval()

In [ ]:
num_pts = 20
# Extract min/max values from the dataset
q1_min, q1_max = torch.min(test_noisy[:,:,0]), torch.max(test_noisy[:,:,0])
p1_min, p1_max = torch.min(test_noisy[:,:,2]), torch.max(test_noisy[:,:,2])
q2_min, q2_max = torch.min(test_noisy[:,:,1]), torch.max(test_noisy[:,:,1])
p2_min, p2_max = torch.min(test_noisy[:,:,3]), torch.max(test_noisy[:,:,3])

# Create 1D grids
q_vals_1 = np.linspace(q1_min, q1_max, num_pts)
p_vals_1 = np.linspace(p1_min, p1_max, num_pts)
q_vals_2 = np.linspace(q2_min, q2_max, num_pts)
p_vals_2 = np.linspace(p2_min, p2_max, num_pts)

# Create a full 4D meshgrid
Q1, P1, Q2, P2 = np.meshgrid(q_vals_1, p_vals_1, q_vals_2, p_vals_2, indexing='ij')

# The resulting arrays are 4D tensors of shape (num_pts, num_pts, num_pts, num_pts)
print(Q1.shape, P1.shape, Q2.shape, P2.shape)  # Should print (num_pts, num_pts, num_pts, num_pts)


In [ ]:
pred_dyn_q1 = np.zeros((num_pts, num_pts, num_pts, num_pts))
pred_dyn_q2 = np.zeros((num_pts, num_pts, num_pts, num_pts))
pred_dyn_p1 = np.zeros((num_pts, num_pts, num_pts, num_pts))
pred_dyn_p2 = np.zeros((num_pts, num_pts, num_pts, num_pts))
true_dyn_q1 = np.zeros((num_pts, num_pts, num_pts, num_pts))
true_dyn_q2 = np.zeros((num_pts, num_pts, num_pts, num_pts))
true_dyn_p1 = np.zeros((num_pts, num_pts, num_pts, num_pts))
true_dyn_p2 = np.zeros((num_pts, num_pts, num_pts, num_pts))
true_h = np.zeros((num_pts, num_pts, num_pts, num_pts))
pred_h = np.zeros((num_pts, num_pts, num_pts, num_pts))

for i in range(num_pts):
    for j in range(num_pts):
        for k in range(num_pts):
            for l in range(num_pts):
                 
                q1 = q_vals_1[i]
                q2 = q_vals_2[j]
                p1 = p_vals_1[k]
                p2 = p_vals_2[l]

                state = torch.tensor([[q1, q2, p1, p2]], dtype=torch.float32)
                # Predicted dynamics using the learned model
                #pred_dyn = forward_ode(state, (model,), (0,)).detach().numpy()

                #pred_dyn_q1[i, j, k, l] = pred_dyn[:, 0]
                #pred_dyn_q2[i, j, k, l] = pred_dyn[:, 1]
                #pred_dyn_p1[i, j, k, l] = pred_dyn[:, 2]
                #pred_dyn_p2[i, j, k, l] = pred_dyn[:, 3]

                # True dynamics
                #true_dyn = henon_heiles(state, None, None).detach().numpy()
                #true_dyn_q1[i, j, k, l] = true_dyn[:, 0]
                #true_dyn_q2[i, j, k, l] = true_dyn[:, 1]
                #true_dyn_p1[i, j, k, l] = true_dyn[:, 2]
                #true_dyn_p2[i, j, k, l] = true_dyn[:, 3]

                # Hamiltonian values
                true_h[i, j, k, l] = (0.5*(p1**2 + p2**2) + 0.5*(q1**2 + q2**2) + (q1**2)*(q2) - ((q2**3)/3))
                pred_h[i, j, k, l] = model(state).squeeze().detach().numpy() - offset
